<a href="https://colab.research.google.com/github/chopstickexe/deep-learning-from-scratch-2/blob/master/ch07_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下準備

$$
\newcommand{\vect}[1]{\mathbf{#1}}
\newcommand{\mat}[1]{\mathbf{#1}}
$$

## 数式の表記

変数: 小文字イタリック $x$

定数: 大文字イタリック $X$

ベクトル: 小文字ローマン体太字 $\vect{x}$

行列: 大文字ローマン体太字 $\mat{X}$

## 公式実装のclone

In [1]:
!git clone --depth=1 https://github.com/oreilly-japan/deep-learning-from-scratch-2.git
import sys 
sys.path.append('deep-learning-from-scratch-2')

Cloning into 'deep-learning-from-scratch-2'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 73 (delta 13), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (73/73), done.


# 7章 RNNによる文章生成

6章で実装した言語モデルが出力する確率分布に従って単語をサンプリングし，文章を生成する．

## 7.1 言語モデルを使った文章生成（`RnnlmGen`，`BetterRnnlmGen`クラス）

In [2]:
import numpy as np
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm):
  def generate(self, start_id, skip_ids=None, sample_size=100): 
    '''
    start_id: 開始単語のID
    skip_ids: <unk>など生成に使いたくない単語のリスト
    sample_size=文長
    '''
    word_ids = [start_id]

    x = start_id
    while len(word_ids) < sample_size:
      x = np.array(x).reshape(1, 1)
      score = self.predict(x)  # Softmaxレイヤの直前の出力
      p = softmax(score.flatten())  # flatten(): ndarrayを一次元の配列化する

      sampled = np.random.choice(len(p), size=1, p=p)  # サンプリング．len(p)は生成されるサンプルの値域最大値，sizeは生成されるサンプル数，pはサンプルする際に使われる確率分布

      if (skip_ids is None) or (sampled not in skip_ids):
        x = sampled
        word_ids.append(int(x))

    return word_ids

PTBデータセットで学習した普通のLSTMモデルを利用して文章生成．

In [3]:
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params('deep-learning-from-scratch-2/ch06/Rnnlm.pkl')

# start文字とskip文字の設定
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 文章生成
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

Done
you continue as a mills of harder such as philip kodak creditors and president.
 this is a port of mr. jones says.
 mr. roman who has registered to rebound after the next couple of years.
 mr. roman also would n't be elected to the staff of its principal.
 it 's a clear effort to treat it left carrying goes taxpayers who also has proved handed.
 london said the government remains reluctant to see that a higher number of boost returns growth futures and options.
 intelogic certain then tuesday 's recreational purchases will be higher


LSTMレイヤを2層にして，各層にDropoutを入れ，さらにEmbeddingの転置行列（H x V, Hは入力単語を表現する隠れベクトルのサイズ，Vは元の語彙数）をAffine Layerの重みにそのまま使うモデル（`ch06/better_rnnlm.py`）に変更すると，生成される文章の質が上がる．
（より言語のルールを守った文章になる）

In [4]:
!curl -o deep-learning-from-scratch-2/ch06/BetterRnnlm.pkl https://www.oreilly.co.jp/pub/9784873118369/BetterRnnlm.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37.7M  100 37.7M    0     0  9697k      0  0:00:03  0:00:03 --:--:-- 9697k


In [5]:
from ch06.better_rnnlm import BetterRnnlm  # LSTMレイヤを2層利用して、各層でDropoutするモデル
from ch07.rnnlm_gen import BetterRnnlmGen  # RnnlmGenの継承クラスをRnnlmからBetterRnnlmに変更しただけのクラス

model = BetterRnnlmGen()
model.load_params('deep-learning-from-scratch-2/ch06/BetterRnnlm.pkl')

# start文字とskip文字の設定
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 文章生成
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

you see the major financial element mr. lane said.
 he adds third-quarter revenue would grow over the next six weeks depending on the issue at the same time.
 overall.
 the company said those totals prior to the explosion that included no single measure of various taxes.
 as yet mr. sung charged the decline 's growth in the first quarter of six years was raising the after-tax profit margins and a loss of c$ of the goodwill contract as the major postal services agents.
 on noon ahead a branch left of his government city consulting firm


## 7.2 seq2seq（足し算問題）

足し算seq2seqを実装するためのデータセットを確認．
Decoder用の開始文字として`_`を入れている． 

In [6]:
from dataset import sequence

(x_train, t_train), (x_test, t_test) = \
  sequence.load_data('addition.txt', seed=1984)
char_to_id, id_to_char = sequence.get_vocab()

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

print(x_train[0])
print(t_train[0])

print(''.join([id_to_char[c] for c in x_train[0]]))
print(''.join([id_to_char[c] for c in t_train[0]]))

(45000, 7) (45000, 5)
(5000, 7) (5000, 5)
[ 3  0  2  0  0 11  5]
[ 6  0 11  7  5]
71+118 
_189 


## 7.3 seq2seqの実装

まずEncoderクラスを実装する．各時刻（各入力）はEmbeddingとLSTMの二層にする．

Encoderクラスの出力として欲しいのは一番最後の時刻が出す隠れベクトルだけなので（ここ重要．最終時刻のセルも要らない），他の時刻のLSTMの隠れベクトルやセルは特に使わない．

In [7]:
class Encoder:
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn

    embed_W = (rn(V, D) / 100).astype('f')
    lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
    lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
    lstm_b = np.zeros(4 * H).astype('f')

    self.embed = TimeEmbedding(embed_W)
    self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False)
    
    self.params = self.embed.params + self.lstm.params
    self.grads = self.embed.grads + self.lstm.grads
    self.hs = None

  def forward(self, xs):
    xs = self.embed.forward(xs)
    hs = self.lstm.forward(xs)  # セルはそもそもforward()の戻り値として返って来ない仕様になっている
    self.hs = hs
    return hs[:, -1, :]  # 最終時刻のhのみ返す

  def backward(self, dh):
    dhs = np.zeros_like(self.hs)
    dhs[:, -1, :] = dh  # 最終時刻のhの勾配をセットする

    dout = self.lstm.backward(dhs)
    dout = self.embed.backward(dout)
    return dout

次にDecoderクラスを実装する．Encoderクラスから最終時刻の隠れベクトルを受け取り，RNNで文章を生成するが，今回は足し算の答えを出したいので，次の時刻の文字はSoftmaxに基づくサンプリングで決めるのではなく，一番高い確率値のものを決定的に出す．

そのため，シンプルなRNNとして実装したEmbedding, LSTM, Affine, SoftmaxWithLossのうち，最後のSoftmaxWithLossを省略して，AffineまでをDecoderとする．（SoftmaxWithLoss部分は後で実装するSeq2seqクラスで扱う）

In [22]:
class Decoder:
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn
    
    embed_W = (rn(V, D) / 100).astype('f')
    lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
    lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
    lstm_b = np.zeros(4 * H).astype('f')
    affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
    affine_b = np.zeros(V).astype('f')

    self.embed = TimeEmbedding(embed_W)
    self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
    self.affine = TimeAffine(affine_W, affine_b)

    self.params, self.grads = [], []

    for layer in (self.embed, self.lstm, self.affine):
      self.params += layer.params
      self.grads += layer.grads

  def forward(self, xs, h):
    self.lstm.set_state(h)

    out = self.embed.forward(xs)
    out = self.lstm.forward(out)
    score = self.affine.forward(out)
    return score

  def backward(self, dscore):
    dout = self.affine.backward(dscore)
    dout = self.lstm.backward(dout)
    dout = self.embed.backward(dout)
    dh = self.lstm.dh
    return dh

  def generate(self, h, start_id, sample_size):
    # Generate a sequence from the given start_id
    sampled = []
    sample_id = start_id
    self.lstm.set_state(h)

    for _ in range(sample_size):
      x = np.array(sample_id).reshape((1, 1))
      out = self.embed.forward(x)
      out = self.lstm.forward(out)
      score = self.affine.forward(out)

      sample_id = np.argmax(score.flatten())  # サンプリングせずに最大値を採用
      sampled.append(int(sample_id))

    return sampled

EncoderクラスとDecoderクラスをつなぎ合わせて，Time Softmax With Lossレイヤで損失を計算する．

In [23]:
from common.base_model import BaseModel

class Seq2seq(BaseModel):
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    V, D, H = vocab_size, wordvec_size, hidden_size
    self.encoder = Encoder(V, D, H)
    self.decoder = Decoder(V, D, H)
    self.softmax = TimeSoftmaxWithLoss()

    self.params = self.encoder.params + self.decoder.params
    self.grads = self.encoder.grads + self.decoder.grads

  def forward(self, xs, ts):
    # xs: 入力列（データセットに入ってる最後の空白文字を除去）
    # ts: 出力列（最初の開始文字を除去）
    decoder_xs, decoder_ts = ts[:, :-1], ts[:, 1:]
    
    h = self.encoder.forward(xs)
    score = self.decoder.forward(decoder_xs, h)
    loss = self.softmax.forward(score, decoder_ts)
    return loss

  def backward(self, dout=1):
    dout = self.softmax.backward(dout)
    dh = self.decoder.backward(dout)
    dout = self.encoder.backward(dh)
    return dout

  def generate(self, xs, start_id, sample_size):
    h = self.encoder.forward(xs)
    sampled = self.decoder.generate(h, start_id, sample_size)
    return sampled


Trainerクラスを使ってSeq2Seqを学習する．

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from common.time_layers import *

# データセットの読み込み
(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
char_to_id, id_to_char = sequence.get_vocab()

# ハイパーパラメータの設定
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

model = Seq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch):
  trainer.fit(x_train, t_train, max_epoch=1, batch_size=batch_size, max_grad=max_grad)

  correct_num = 0
  for i in range(len(x_test)):
    question, correct = x_test[[i]], t_test[[i]]
    verbose = i < 10
    correct_num += eval_seq2seq(model, question, correct, id_to_char, verbose)

  acc = float(correct_num) / len(x_test)
  acc_list.append(acc)
  print('val acc %.3f%%' % (acc * 100))

| epoch 1 |  iter 1 / 351 | time 0[s] | loss 2.56
| epoch 1 |  iter 21 / 351 | time 1[s] | loss 2.53
| epoch 1 |  iter 41 / 351 | time 2[s] | loss 2.17
| epoch 1 |  iter 61 / 351 | time 3[s] | loss 1.96
| epoch 1 |  iter 81 / 351 | time 5[s] | loss 1.92
| epoch 1 |  iter 101 / 351 | time 6[s] | loss 1.87
| epoch 1 |  iter 121 / 351 | time 7[s] | loss 1.85
| epoch 1 |  iter 141 / 351 | time 8[s] | loss 1.83
| epoch 1 |  iter 161 / 351 | time 10[s] | loss 1.79
| epoch 1 |  iter 181 / 351 | time 11[s] | loss 1.77
| epoch 1 |  iter 201 / 351 | time 13[s] | loss 1.77
| epoch 1 |  iter 221 / 351 | time 14[s] | loss 1.76
| epoch 1 |  iter 241 / 351 | time 15[s] | loss 1.76
| epoch 1 |  iter 261 / 351 | time 16[s] | loss 1.76
| epoch 1 |  iter 281 / 351 | time 18[s] | loss 1.75
| epoch 1 |  iter 301 / 351 | time 19[s] | loss 1.74
| epoch 1 |  iter 321 / 351 | time 20[s] | loss 1.75
| epoch 1 |  iter 341 / 351 | time 22[s] | loss 1.74
Q 77+85  
T 162 
☒ 100 
---
Q 975+164
T 1139
☒ 1000
---
Q 58